In [90]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Activation, Dense
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

In [21]:
dataset = 'english.txt'

In [22]:
f = open(dataset, 'r')
data = f.read().strip()
chars = set(data)
print(chars, len(chars))
index_to_char = dict([(i,e) for i, e in enumerate(chars)])
char_to_index = dict([(e, i) for i, e in enumerate(chars)])
print(index_to_char)

{' ', 'e', 'q', 'u', 'x', 'f', 'w', 'j', 's', 'l', 'a', 'g', 'y', 'r', '.', 'h', '\n', 'p', 'z', 'o', 't', 'd', 'k', 'n', 'c', 'b', 'm', 'v', 'i'} 29
{0: ' ', 1: 'e', 2: 'q', 3: 'u', 4: 'x', 5: 'f', 6: 'w', 7: 'j', 8: 's', 9: 'l', 10: 'a', 11: 'g', 12: 'y', 13: 'r', 14: '.', 15: 'h', 16: '\n', 17: 'p', 18: 'z', 19: 'o', 20: 't', 21: 'd', 22: 'k', 23: 'n', 24: 'c', 25: 'b', 26: 'm', 27: 'v', 28: 'i'}


In [23]:
NCHARS = len(chars)
MAXLEN_SENTENCE = 70

In [106]:
# create a list of sentences and the next char after that sentence
sentences = []
nextchars = []

for i in range(0, len(data)-MAXLEN_SENTENCE-1):
    sentences.append(data[i:i+MAXLEN_SENTENCE])
    nextchars.append(data[i + MAXLEN_SENTENCE])
    
    if(len(sentences) == 20000):
        break
NSENTENCES = len(sentences)
print(len(sentences))
print(len(sentences[77]))

20000
70


In [107]:
trainX = np.zeros((NSENTENCES, MAXLEN_SENTENCE, NCHARS), dtype=np.bool)
trainY = np.zeros((NSENTENCES, NCHARS), dtype=np.bool)

for i in range(NSENTENCES):
    for j, e in enumerate(sentences[i]):
        trainX[i][j][char_to_index[e]] = 1
    trainY[i][char_to_index[nextchars[i]]] = 1
    
np.save('trainX', trainX)
np.save('trainY', trainY)

In [111]:
model = Sequential()
model.add(LSTM(256, input_shape = trainX.shape[1:], return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(NCHARS))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_59 (LSTM)               (None, 70, 256)           292864    
_________________________________________________________________
dropout_34 (Dropout)         (None, 70, 256)           0         
_________________________________________________________________
lstm_60 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_35 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 29)                7453      
_________________________________________________________________
activation_32 (Activation)   (None, 29)                0         
Total params: 825,629
Trainable params: 825,629
Non-trainable params: 0
_________________________________________________________________
None

In [112]:
filepath="weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(trainX, trainY, batch_size=64, validation_split=0.3, epochs=1, callbacks=callbacks_list)

Train on 14000 samples, validate on 6000 samples
Epoch 1/1
11200/14000 [=======================>......] - ETA: 10s - loss: 0.4330 - acc: 0.8701

In [175]:
def predictNextChar(sampleText, start):
#     print(sampleText)
    x = np.zeros((1, MAXLEN_SENTENCE, NCHARS), dtype=bool)
    for i, c in enumerate(sampleText[start:start+MAXLEN_SENTENCE]):
        x[0][i][char_to_index[c]] = 1
#     predicted = model.predict(x)
#     print(np.argmax(predicted[0]))
    y = index_to_char[np.argmax(model.predict(x)[0])]
    return y
    
def generateText(seed):
    result = str(seed)
    print(len(seed))
    print('Seed: ', seed)
    print('-'*50)
    i = 0
    maxlen = 350
    while len(result) < maxlen:
        result += predictNextChar(result, i)
        i += 1
    return result
print(generateText(sentences[16366]))
# print(predictNextChar(sentences[2], 0))

70
Seed:  he towering height of mount sinai saying hold firmly to what we have g
--------------------------------------------------
he towering height of mount sinai saying hold firmly to what we have given you and bring ever and earen and in to us ara fall things then he placed them what allah hath revealed to you that they are to meet their lord and that they are to meet their lord and that they are to meet their lord and that they are to meet their lord and that they are to 
